In [ ]:
import os
import numpy
import pandas as pd

In [ ]:
train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 0.1

In [ ]:
mgh_mrns = list(map(lambda filename: filename[:-4], os.listdir('/storage/shared/ecg/mgh')))
bwh_mrns = list(map(lambda filename: filename[:-4], os.listdir('/storage/shared/ecg/bwh')))

if 'bad_mrn' in mgh_mrns:
    mgh_mrns.remove('bad_mrn')
if 'bad_mrn' in bwh_mrns:
    bwh_mrns.remove('bad_mrn')

mgh_mrns = list(map(int, mgh_mrns))
bwh_mrns = list(map(int, bwh_mrns))

ecg_mrns = set(mgh_mrns) | set(bwh_mrns)

In [ ]:
c3po = pd.read_csv('~/dropbox/c3po/c3po-formatted-mrn.csv')
c3po_mrns = set(c3po['mrn'])

In [ ]:
echo = set(pd.read_csv('~/dropbox/ecgnet-as/data/echo.csv')['mrn'])
sts = set(pd.read_csv('~/dropbox/sts-data/sts-mgh.csv')['medrecn'])
ici = set(pd.read_csv('~/dropbox/ecg-ici/data/ici.csv')['mgh_mrn'])
mrns_to_remove = echo | sts | ici

In [ ]:
overlapped = set(mgh_mrns) & set(bwh_mrns)

In [ ]:
overlapped

In [ ]:
mrns = (c3po_mrns & ecg_mrns) - mrns_to_remove

In [ ]:
c3po = c3po[c3po['mrn'].isin(mrns)]

In [ ]:
c3po_lived = c3po[c3po['death_date'].isnull()]
c3po_died = c3po[~c3po['death_date'].isnull()]

In [ ]:
c3po.sample(frac=1, random_state=0)

l_tv_idx = int(train_ratio * len(c3po_lived))
l_vt_idx = int((train_ratio + valid_ratio) * len(c3po_lived))

d_tv_idx = int(train_ratio * len(c3po_died))
d_vt_idx = int((train_ratio + valid_ratio) * len(c3po_died))

train = pd.concat([c3po_lived[:l_tv_idx], c3po_died[:d_tv_idx]])
valid = pd.concat([c3po_lived[l_tv_idx:l_vt_idx], c3po_died[d_tv_idx:d_vt_idx]])
test = pd.concat([c3po_lived[l_vt_idx:], c3po_died[d_vt_idx:]])

In [ ]:
train['death_date'].isnull().value_counts() / len(train)

In [ ]:
valid['death_date'].isnull().value_counts() / len(valid)

In [ ]:
test['death_date'].isnull().value_counts() / len(test)

In [ ]:
train['mrn'].sample(frac=1, random_state=0).to_csv('~/dropbox/c3po/train.csv', index=False)
valid['mrn'].sample(frac=1, random_state=0).to_csv('~/dropbox/c3po/valid.csv', index=False)
test['mrn'].sample(frac=1, random_state=0).to_csv('~/dropbox/c3po/test.csv', index=False)